In [1]:
!pip install torch transformers scikit-learn pandas


In [3]:
import os
import re
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [5]:
CSV_PATH   = "spam.csv"              # <-- change to your dataset path
MODEL_NAME = "bert-base-uncased"
TEXT_MAXLEN = 256
BATCH_SIZE  = 16
EPOCHS      = 3
LR          = 2e-5
SEED        = 42

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


In [9]:
df = pd.read_csv(r"C:\Users\Hp\Downloads\spam_ham_dataset.csv")
df

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [10]:
# Try to find a text column
text_candidates = ["text", "message", "email", "content", "body", "subject", "sms", "mail", "review_text"]
label_candidates = ["label", "spam", "target", "class", "category", "is_spam", "isSpam"]

def pick_column(candidates, columns):
    cols_lower = {c.lower(): c for c in columns}
    for c in candidates:
        if c in cols_lower:
            return cols_lower[c]

In [18]:
# fallback: longest text-like column
  obj_cols = [c for c in df.columns if df[c].dtype == "object"]
    if obj_cols:
        # choose the one with largest average length
        avg_lens = [(c, df[c].astype(str).str.len().mean()) for c in obj_cols]
        avg_lens.sort(key=lambda x: x[1], reverse=True)
        return avg_lens[0][0]
    return columns[0]

TEXT_COL  = pick_column(text_candidates, df.columns)
LABEL_COL = pick_column(label_candidates, df.columns)

print(f"Using TEXT_COL='{TEXT_COL}', LABEL_COL='{LABEL_COL}'")


IndentationError: unexpected indent (79096135.py, line 2)